In [16]:
import os
import numpy as np
from PIL import Image
from IPython.display import display

# ───────────────────── helpers ─────────────────────
def sigmoid(x):
    return 1 / (1 + np.exp(-x, dtype=np.float32))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)
def mse_cost(output, target):
    return np.mean((output - target) ** 2)

# ───────────────────── core classes ─────────────────────
class Layer:
    """Stores the activations for one layer."""
    def __init__(self, size: int):
        self.size = size
        self.data = None              # will hold (batch, size)
        self.z = None     # Weighted sum input (for derivative)


class Weight:
    """Random weight matrix connecting two layers (fan_in × fan_out)."""
    def __init__(self, fan_in: int, fan_out: int):
        self.val = np.random.randn(fan_in, fan_out).astype(np.float32)

class Bias:
    """Bias row vector for a layer (1 × fan_out)."""
    def __init__(self, fan_out: int):
        self.val = np.random.randn(1, fan_out).astype(np.float32)

class NeuralNet:
    def __init__(self, activation, activation_deriv, lr=0.01):
        self.act        = activation          # forward activation
        self.act_deriv  = activation_deriv    # derivative for back‑prop
        self.lr         = lr                  # learning‑rate
        self.layers, self.weights, self.biases = [], [], []

    def add_layer(self, size: int):
        """Append a new layer; create Weight/Bias if not the first."""
        self.layers.append(Layer(size))
        if len(self.layers) > 1:
            fan_in  = self.layers[-2].size
            fan_out = self.layers[-1].size
            self.weights.append(Weight(fan_in, fan_out))
            self.biases.append(Bias(fan_out))

    def forward(self):
        """Run activations from layer-0 through the last layer."""
        for i in range(1, len(self.layers)):
            x = self.layers[i - 1].data                # previous activations
            z = x @ self.weights[i - 1].val + self.biases[i - 1].val
            self.layers[i].z    = z                # store weighted sum for back-prop
            self.layers[i].data = self.act(z)

    def backward(self, targets):
        deltas = [None] * len(self.layers)

        # 🔻 Output delta
        output = self.layers[-1].data
        z = self.layers[-1].z
        deltas[-1] = (output - targets) * self.act_deriv(z)

        # 🔻 Hidden deltas
        for i in reversed(range(1, len(self.layers) - 1)):
            z = self.layers[i].z
            w_next = self.weights[i].val
            delta_next = deltas[i + 1]
            deltas[i] = (delta_next @ w_next.T) * self.act_deriv(z)

        # 🔻 Gradient Descent update
        for i in range(len(self.weights)):
            a_prev = self.layers[i].data
            dw = a_prev.T @ deltas[i + 1]
            db = np.sum(deltas[i + 1], axis=0, keepdims=True)
            self.weights[i].val -= self.lr * dw
            self.biases[i].val -= self.lr * db


    @property
    def output(self):
        return self.layers[-1].data

# ───────────────────── load MNIST csv ─────────────────────
folder = "mnist_csv"                        
csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No CSV files found in", folder)

csv_path = os.path.join(folder, csv_files[0])
data   = np.genfromtxt(csv_path, delimiter=",", skip_header=1)
labels = data[:, 0].astype(int)
pixels = data[:, 1:].astype(np.uint8)

# # Prepare batch
n = 100
indices = np.random.choice(len(labels), n, replace=False)
images = pixels[indices].astype(np.float32) / 255.0
true_labels = labels[indices]

# One-hot encode targets
targets = np.zeros((n, 10), dtype=np.float32)
for i in range(n):
    targets[i, true_labels[i]] = 1.0



# ───────────────────── pick & show one image ─────────────────────
max_idx = len(labels) - 1
idx     = int(input(f"Enter image index (0–{max_idx}): "))
if not 0 <= idx <= max_idx:
    raise ValueError("Index out of range")

img_arr = pixels[idx].reshape(28, 28)
display(Image.fromarray(img_arr, mode="L"))
print("True label:", labels[idx])

 

True label: 1


In [ ]:
net = NeuralNet(sigmoid, sigmoid_deriv, lr=0.05)
net.add_layer(784)
net.add_layer(107)
net.add_layer(26)
net.add_layer(10)

epochs = 100  # or 50, 200 etc.

for epoch in range(epochs):
    net.layers[0].data = images        # load batch into input layer
    net.forward()
    cost = mse_cost(net.output, targets)
    predictions = np.argmax(net.output, axis=1)
    accuracy = np.mean(predictions == true_labels)
    print(f"Epoch {epoch + 1:3d}: Cost = {cost:.6f} | Accuracy = {accuracy:.2%}")
    net.backward(targets)

predicted_labels = np.argmax(net.output, axis=1)

print("\n🎯 Final Predictions After Training:")
print("Predicted labels:", predictions.tolist())
print("True labels     :", true_labels.tolist())
print("Final cost      :", round(cost, 6))

Raw output (before training):
[[9.86782432e-01 2.66093135e-01 1.73811615e-01 1.04510486e-01
  2.26362497e-02 6.51445448e-01 1.14946077e-02 1.53052760e-02
  4.11456615e-01 9.97583151e-01]
 [9.03506696e-01 2.82021195e-01 2.16135550e-02 9.14232790e-01
  6.39301538e-02 9.48962986e-01 8.29897879e-04 1.86561216e-02
  8.90658259e-01 8.21099520e-01]
 [9.93570089e-01 4.77614030e-02 3.87046598e-02 5.39100647e-01
  1.14554122e-01 7.61058390e-01 3.29228467e-03 2.57533789e-02
  3.12187582e-01 9.94052231e-01]
 [9.84395862e-01 1.17097393e-01 4.46199533e-03 5.59541881e-01
  4.86307265e-03 9.16437805e-01 9.98156145e-04 1.44843236e-01
  5.56324303e-01 8.64940286e-01]
 [9.92045760e-01 7.97397733e-01 6.42777532e-02 2.34788135e-01
  4.23485786e-02 6.92069650e-01 1.31845754e-03 8.31218064e-03
  2.33595327e-01 9.96314228e-01]
 [9.93024588e-01 1.83759723e-02 7.82338381e-02 1.53958067e-01
  7.38790259e-02 9.92865324e-01 6.31766859e-03 7.46057648e-03
  8.96895766e-01 9.95493174e-01]
 [9.64673638e-01 3.72071750e